#install the required libaries

!pip install git+ssh://git@github.com/healx/healx-chem.git && pip install git+ssh://git@github.com/healx/healx-python.git#egg=healx && pip insyall healx[graphql]

In [1]:
from healx.graphql_client import client
from healx_chem import CompoundSpace
import csv
import pandas as pd
import os
from dotenv import load_dotenv
#from pathlib import Path 

2024-09-19 12:26:29 INFO: NumExpr defaulting to 8 threads. [numexpr.utils:161]


In [2]:
# Load the environment variables from the .env file
load_dotenv(verbose = True)

# Access the environment variables
eee = os.getenv('GRAPHQL_API_URL')
os.getenv('GRAPHQL_API_CLIENT')
os.getenv('GRAPHQL_API_KEY')

2024-09-19 12:26:31 INFO: Python-dotenv could not find configuration file .env. [dotenv.main:77]


In [3]:
#get the tautomer parent SMILES for all compounds in KB
def fetch_smiles_KB():
    query = """{
      compounds{
        name
        primaryForm{
          standardisedStructures{
            tautomerParentSmiles
          }
        }
      }
    }
"""
    compounds = client(query)["compounds"]
    return compounds

In [4]:
KBcompounds = fetch_smiles_KB()

HTTPError: 403 Client Error: FORBIDDEN for url: https://kb.healx.io/api/graphql

In [16]:
KBcompounds[:10]

[{'name': 'PIFOXIME',
  'primaryForm': {'standardisedStructures': [{'tautomerParentSmiles': 'C/C(=N\\O)c1ccc(OCC(=O)N2CCCCC2)cc1'}]}},
 {'name': 'CHEMBL2089269',
  'primaryForm': {'standardisedStructures': [{'tautomerParentSmiles': 'Cn1c(Sc2ncnc3sccc23)nnc1C1CC1'},
    {'tautomerParentSmiles': 'Cn1c(Sc2ncnc3sccc23)nnc1C1CC1'}]}},
 {'name': 'AD-204', 'primaryForm': {'standardisedStructures': []}},
 {'name': 'CA-01X', 'primaryForm': {'standardisedStructures': []}},
 {'name': 'JCP2022_093453',
  'primaryForm': {'standardisedStructures': [{'tautomerParentSmiles': 'CN(C)CC1CN(S(C)(=O)=O)CC1NC(=O)c1cscn1'}]}},
 {'name': 'JCP2022_093454',
  'primaryForm': {'standardisedStructures': [{'tautomerParentSmiles': 'O=c1cc(CN2CCCC2CC(O)c2ccccc2)nc2ccccn12'}]}},
 {'name': 'MURODERMIN', 'primaryForm': {'standardisedStructures': []}},
 {'name': 'FASINUMAB', 'primaryForm': {'standardisedStructures': []}},
 {'name': 'RUNX1/CBF beta inhibitors, Interprotein',
  'primaryForm': {'standardisedStructures': []}

In [17]:
KBsmiles = {}
multiple_parents = []
for dic in KBcompounds:
    for key, value in dic.items():
        smiles_list = []
        if key == "name":
            name = value
        if key == "primaryForm":
            for k, v in value.items():
                if len(v)>1:
                    for i in v:
                        for taut, smiles in i.items():
                            if smiles not in smiles_list:
                                smiles_list.append(smiles)
# get list of compounds with multiple tautomer parent structures       
        if len(smiles_list)>1:
            temp_dic = {}
            temp_dic[name]=smiles_list
            multiple_parents.append(temp_dic)
# retain only compound name and first tautomer parent smiles
        if len(smiles_list):
            KBsmiles[name] = smiles_list[0]

print(len(KBsmiles))
KBsmiles

58824


{'CHEMBL2089269': 'Cn1c(Sc2ncnc3sccc23)nnc1C1CC1',
 'CHEMBL2359269': 'COc1ccc(CCC(=O)NCc2ccc3c(c2)CCN3C(=O)c2ccccc2)cc1',
 'CHEMBL1453901': 'O=C(c1ccccc1)N(Cc1ccccc1)Cc1ccccc1',
 'CHEMBL3451805': 'CCC(CC)c1cc(C(=O)N2CCN(C(C)C(N)=O)CC2)on1',
 'EPINEPHRINE': 'CNC[C@H](O)c1ccc(O)c(O)c1',
 'HERNIARIN': 'COc1ccc2ccc(=O)oc2c1',
 'CHEMBL56835': 'CC(=O)NC1(c2ccccc2)CCN(CCC(CN(C)C(=O)c2ccccc2)c2ccc(Cl)c(Cl)c2)CC1',
 '6-THIOGUANYLIC ACID': 'Nc1nc(=S)c2ncn([C@@H]3O[C@H](COP(=O)(O)O)[C@@H](O)[C@H]3O)c2[nH]1',
 'DEXAMETHASONE PHOSPHORIC ACID': 'C[C@@H]1C[C@H]2[C@@H]3CC=C4CC(=O)C=C[C@]4(C)[C@@]3(F)[C@@H](O)C[C@]2(C)[C@@]1(O)C(=O)COP(=O)(O)O',
 'CLORPRENALINE': 'CC(C)NCC(O)c1ccccc1Cl',
 'TRIBROMSALAN': 'O=C(Nc1ccc(Br)cc1)c1cc(Br)cc(Br)c1O',
 'CHEMBL1388161': 'COc1ccc(S(=O)(=O)NCCOc2ccccc2)cc1C',
 'CHEMBL1518888': 'O=C(COC(=O)c1cc(-c2ccco2)nc2ccccc12)NCc1ccccc1',
 'PIPENZOLATE (ChEMBL CHEMBL1619528)': 'CC[N+]1(C)CCCC(OC(=O)C(O)(c2ccccc2)c2ccccc2)C1',
 'AMPHOMYCIN': 'CCC(C)CCCCC/C=C/CC(=O)NC(CC(=O)O)C(

In [18]:
cs = CompoundSpace(KBsmiles)
cs.calculate_fingerprints()

Calculating fingerprints: 100%|██████████| 58824/58824 [00:09<00:00, 5923.03it/s]


In [47]:
KBsmiles["SULFINPYRAZONE"]
print(cs)

CompoundSpace:
	58824 valid compounds
	0 invalid compounds


In [19]:
#get ID, name and structure for Dotmatics compound not mapping to KB
# Open the CSV file
with open("/Users/sonialiggi/Library/CloudStorage/GoogleDrive-sonia.liggi@healx.io/Shared drives/Internal Projects/Dotmatics/KB Mapping/NotMapping.csv", "r") as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Skip the header row (optional)
    next(reader)

    # Create an empty dictionary
    data_dict = {}

    # Iterate over each row in the CSV file
    for row in reader:
        # Get the values from the columns
        ID = row[0]
        name = row[1]
        smi = row[2]
        data_dict[ID] = [name, smi]
    
# Print the dictionary
print(len(data_dict))

159


In [27]:
print(data_dict["HLX0000186"])

['5,7-Dichloro-8-hydroxyquinoline beta-D-glucuronide', 'O[C@@H]1[C@@H](O)[C@@H](O[C@@H]([C@H]1O)C(=O)O)Oc1c(cc(c2cccnc12)Cl)Cl']


In [9]:

pp = cs.search(query=data_dict['HLX0001904'][1])
filtered_matrix = pp.filter(pp["ref"] ==0)
pp
"""filtered_matrix =filtered_matrix.reset_index().rename(columns={"index": "KB_name", "ref": "similarity_score"})
filtered_matrix["DotmaticsID"] = "HLX0001904"
filtered_matrix["Dotmatics_smiles"] = data_dict["HLX0001904"][1]
filtered_matrix["KB_smiles"] = filtered_matrix["KB_name"].map(KBsmiles)
filtered_matrix["Dotmatics_name"] = data_dict["HLX0001904"][0]
filtered_matrix = filtered_matrix[["DotmaticsID", "Dotmatics_name", "Dotmatics_smiles", "KB_name", "KB_smiles", "similarity_score"]]
filtered_matrix"""


NameError: name 'cs' is not defined

In [28]:
#run similarity search
dataframes = []
c = 1
for id, llist in data_dict.items():
    query_smiles = llist[1]
    sim_matrix = cs.search(query=query_smiles)
    filtered_matrix = sim_matrix[sim_matrix["ref"] == sim_matrix["ref"].max()]
    filtered_matrix = filtered_matrix.reset_index().rename(columns={"index": "KB_name", "ref": "similarity_score"})
    filtered_matrix["DotmaticsID"] = id
    filtered_matrix["index"] = c
    filtered_matrix["Dotmatics_smiles"] = llist[1]
    filtered_matrix["KB_smiles"] = filtered_matrix["KB_name"].map(KBsmiles)
    filtered_matrix["Dotmatics_name"] = data_dict[id][0]
    filtered_matrix = filtered_matrix[["index", "DotmaticsID", "Dotmatics_name", "Dotmatics_smiles", "KB_name", "KB_smiles", "similarity_score"]]
    dataframes.append(filtered_matrix)
    c +=1
result_df = pd.concat(dataframes, ignore_index=True)


HLX0000169


Building matrix: 100%|██████████| 1/1 [00:00<00:00,  6.73it/s]


HLX0000186


Building matrix: 100%|██████████| 1/1 [00:00<00:00,  3.82it/s]


HLX0000191


...

Building matrix: 100%|██████████| 1/1 [00:00<00:00,  7.20it/s]


HLX0002443


Building matrix: 100%|██████████| 1/1 [00:00<00:00,  7.28it/s]


HLX0002444


Building matrix: 100%|██████████| 1/1 [00:00<00:00,  7.48it/s]


In [29]:
result_df

,index,DotmaticsID,Dotmatics_name,Dotmatics_smiles,KB_name,KB_smiles,similarity_score
0,1,HLX0000169,Peficitinib,NC(=O)c1cnc2[nH]ccc2c1NC1[C@@H]2CC3C[C@H]1CC(O...,PEFICITINIB,NC(=O)c1cnc2[nH]ccc2c1N[C@@H]1[C@@H]2CC3C[C@H]...,1.000000
1,1,HLX0000169,Peficitinib,NC(=O)c1cnc2[nH]ccc2c1NC1[C@@H]2CC3C[C@H]1CC(O...,CHEMBL4238926,NC(=O)c1cnc2[nH]ccc2c1NC1C2CC3CC1CC(O)(C3)C2,1.000000
2,2,HLX0000186,"5,7-Dichloro-8-hydroxyquinoline beta-D-glucuro...",O[C@@H]1[C@@H](O)[C@@H](O[C@@H]([C@H]1O)C(=O)O...,CHEMBL1528425,O=C(Oc1c(Cl)cc(Cl)c2cccnc12)c1ccco1,0.491228
3,3,HLX0000191,5-Chloroquinoline,Clc1cccc2ncccc12,CLOXYQUIN,Oc1ccc(Cl)c2cccnc12,0.468750
4,4,HLX0000224,Dothiepin-d3,[2H]c1cc2c(c(c1[2H])[2H])/C(=C/CCN(C)C)c1ccccc...,DOTHIEPIN (ChEMBL CHEMBL1492500),CN(C)CC/C=C1\c2ccccc2CSc2ccccc21,0.755556
...,...,...,...,...,...,...,...
235,156,HLX0002440,Oligomycin A + Antimycin A Combination,CCCCCC[C@@H]1[C@@H](OC(=O)CC(C)C)[C@H](C)OC(=O...,Oligomycin A (LINCS LSM-43272),CCC1/C=C/C=C/C[C@H](C)[C@@H](O)[C@](C)(O)C(=O)...,0.609023
236,156,HLX0002440,Oligomycin A + Antimycin A Combination,CCCCCC[C@@H]1[C@@H](OC(=O)CC(C)C)[C@H](C)OC(=O...,Oligomycin-A (LINCS LSM-43347),CC[C@@H]1/C=C/C=C/C[C@H](C)[C@@H](O)[C@](C)(O)...,0.609023
237,157,HLX0002442,,[3H]c1cc(c2cccnc2c1O)[N+](=O)[O-],NITROXOLINE,O=[N+]([O-])c1ccc(O)c2ncccc12,0.771429
238,158,HLX0002443,Nitroxoline sulfate triethylamine,[O-]S(=O)(=O)Oc1ccc(c2cccnc12)N(=O)=O,NITROXOLINE,O=[N+]([O-])c1ccc(O)c2ncccc12,0.571429


In [30]:
result_df.to_csv("/Users/sonialiggi/Library/CloudStorage/GoogleDrive-sonia.liggi@healx.io/Shared drives/Internal Projects/Dotmatics/KB Mapping/NotMapping_KBsimilarity.csv", index = False)
